Для запуска нужно поставить:

- MXNet. See the instructions for your operating system in [Setup and Installation](http://mxnet.io/install/index.html)

- [Python Requests](http://docs.python-requests.org/en/master/), [Matplotlib](https://matplotlib.org/) and [Jupyter Notebook](http://jupyter.org/index.html).

Также потребуются numpy, cv2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mxnet as mx
import os
import glob
import time
from collections import namedtuple

Далее описаны две функции. Первая помогает выкачивать изображение и обрабатывать его, вторая собственно предсказывает результат модели на изображении. Она сохраняет 5 наиболее вероятных классов, к которым может относиться изображение, и вероятности этих классов.

In [3]:
Batch = namedtuple('Batch', ['data'])

def get_image(url, show=False, local=False):
    # download and show the image
    if (local == False):
        fname = mx.test_utils.download(url, dirname='images')
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url, show=False, local=False):
    res = []
    img = get_image(url, show=show, local=local)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        res.append([prob[i], labels[i]])
        #print('probability=%f, class=%s' %(prob[i], labels[i]))
    return res

Я выкачал 4 предобученные модели и протестировал их на руками собранных выборках картинок ручек, ключей, телефонов. Результаты:

In [4]:
models = ['Inception', 'caffenet', 'nin', 'squeezenet_v1.0']
number = [9, 0, 0, 0]
for i, name in enumerate(models):
    #Initializing model
    print('Model ' + name)
    sym, arg_params, aux_params = mx.model.load_checkpoint(name + '/' + name, number[i])
    mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
    mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
             label_shapes=mod._label_shapes)
    mod.set_params(arg_params, aux_params, allow_missing=True)
    with open('synset.txt', 'r') as f:
        labels = [l.rstrip() for l in f]
    #Predict testing
    datasets = ['pen', 'key', 'phone']
    sz = 0
    start = time.time()
    for i, dataset in enumerate(datasets):
        ans = np.zeros(5)
        cnt = np.zeros(6)
        for filename in glob.iglob('images/' + dataset + '/*'):
            result = predict(filename,  local=True)
            was = False
            for tmp in result:
                prob = tmp[0]
                tag = tmp[1].lower()
                if (tag.find(dataset) != -1):
                    ans[i] += prob
                    cnt[i] += 1
                    was = True
                    break
            if (not was):
                cnt[5] += 1
        print('For dataset ' + dataset + ':')
        for i in range(5):
            if (cnt[i] > 0):
                print('Recognised with ' + str(i + 1) + ' priority: ' + str(cnt[i]) + 
                  ' images with average probability =  ' + str(ans[i] / cnt[i]))
            else:
                print('Recognised with ' + str(i + 1) + ' priority: ' + str(cnt[i]))
        print('Not recognized: ', cnt[5])
        sz += np.sum(cnt)
    end = time.time()
    print(str(sz) + ' images were analyzed in ' + str(end - start) + ' seconds.')
    print

Model Inception
For dataset pen:
Recognised with 1 priority: 23.0 images with average probability =  0.22384909503971753
Recognised with 2 priority: 0.0
Recognised with 3 priority: 0.0
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 9.0)
For dataset key:
Recognised with 1 priority: 0.0
Recognised with 2 priority: 17.0 images with average probability =  0.23664229659034924
Recognised with 3 priority: 0.0
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 19.0)
For dataset phone:
Recognised with 1 priority: 0.0
Recognised with 2 priority: 0.0
Recognised with 3 priority: 5.0 images with average probability =  0.14087858721613883
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 27.0)
100.0 images were analyzed in 114.897554874 seconds.

Model caffenet
For dataset pen:
Recognised with 1 priority: 0.0
Recognised with 2 priority: 0.0
Recognised with 3 priority: 0.0
Recognised with 4 prio